In [3]:
from langchain_groq import ChatGroq
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display

In [ ]:
groq_api_key = 'YOUR_GROQ_API_KEY'
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    sentiment: str

# 1. Preprocessing Node
def preprocess(state: State) -> State:
    cleaned = state["messages"][-1].content.strip()
    state["messages"][-1].content = cleaned
    return state

# 2. Sentiment Analysis Node
def analyze_sentiment(state: State) -> State:
    msg = state["messages"][-1].content
    state["sentiment"] = "positive" if "good" in msg else "neutral"
    return state

# 3. Chatbot Node
def chatbot(state: State) -> State:
    return {"messages": llm.invoke(state['messages'])}

# 4. Logging Node
def logger(state: State) -> State:
    print(f"LOG: {state['messages'][-1].content}, Sentiment: {state.get('sentiment')}")
    return state

In [6]:
# Build the graph
builder = StateGraph(State)
builder.add_node("preprocess", preprocess)
builder.add_node("analyze_sentiment", analyze_sentiment)  # renamed
builder.add_node("chatbot", chatbot)
builder.add_node("logger", logger)

In [7]:
# Define flow
builder.add_edge(START, "preprocess")
builder.add_edge("preprocess", "analyze_sentiment")  # renamed
builder.add_edge("analyze_sentiment", "chatbot")     # renamed
builder.add_edge("chatbot", "logger")
builder.add_edge("logger", END)

In [8]:
graph = builder.compile()

In [9]:
# Draw the graph
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [10]:
# Invoke the graph
input_message = "This is a good test message."
final_state = graph.invoke({"messages": ("user", input_message)})

# Print the final state
print("Final State:", final_state)
print("Chatbot's response:", final_state['messages'][-1].content)
print("Detected Sentiment:", final_state['sentiment'])

input_message_2 = "This is a neutral message."
final_state_2 = graph.invoke({"messages": ("user", input_message_2)})
print("Final State 2:", final_state_2)
print("Chatbot's response 2:", final_state_2['messages'][-1].content)
print("Detected Sentiment 2:", final_state_2['sentiment'])

LOG: I understand! Is there anything specific you'd like me to do with this test message? 

For example, would you like me to:

* **Analyze the sentiment?** (I can tell you if it's positive, negative, or neutral)
* **Rewrite it in a different style?** (e.g., more formal, more casual)
* **Translate it into another language?**
* **Generate a response?**


Let me know how I can help! 😊  

, Sentiment: positive
Final State: {'messages': [HumanMessage(content='This is a good test message.', additional_kwargs={}, response_metadata={}, id='52a3fd9b-b62b-4253-bfab-bec666c94a66'), AIMessage(content="I understand! Is there anything specific you'd like me to do with this test message? \n\nFor example, would you like me to:\n\n* **Analyze the sentiment?** (I can tell you if it's positive, negative, or neutral)\n* **Rewrite it in a different style?** (e.g., more formal, more casual)\n* **Translate it into another language?**\n* **Generate a response?**\n\n\nLet me know how I can help! 😊  \n\n", add